# QMCS notebook
QMCS is a package to perform monte Carlo analysis of quantum networks. Specifically it focuses on the distribution multipartite GHZ state, however, the simulations can also be used for the two user case.
We use model network topologies as graph. As such topologies are considered as networkx graphs. 
A default 6x6 grid topology is assumed.

In [ ]:
import matplotlib.pyplot as plt
import networkx as nx
graph = nx.grid_2d_graph(6,6) 
plt.figure(figsize = (4,4))
nx.draw(graph,
        with_labels=True,
        pos = {(x, y): (y, x) for x, y in graph.nodes()})
plt.show()

In [ ]:
from qmcs.src.simulation_helpers import add_default_network_attributes,get_best_source_and_star,steiner_tree,centroid_grid
from qmcs.src.plotting_helpers import plot_graph_routing_solution
# example of star and tree single-path routing 
add_default_network_attributes(graph)
users = [(3,5),(3,1),(4,4),(1,1)]
source_node, routing_solution_star, _ = get_best_source_and_star(
            graph =graph,
            users = users,
            weight="p_edge_log",
            source_node_first_guess=centroid_grid(users),
        )
# Note that it can be more efficent for a user to operate as the centre node!
plot_graph_routing_solution(graph=graph,
                            routing_solution = routing_solution_star,
                            users = users,
                            source = source_node)

routing_solution_tree = steiner_tree(graph,users,weight = 'p_edge_log')
plot_graph_routing_solution(graph=graph,
                            routing_solution = routing_solution_tree,
                            users = users,
                            source = None)

Multi-path routing uses a similar approach, but attempts routing with the dynamic link-state graph G'. This is the set of edges that hold a Bell state during a given timeslot.\
for example, in a graph in which edges are generated with probability 0.7, and have edge-weight 1, we can look at random instances of G' to see if routing is possible

In [ ]:
import random
# Multi-path routing uses a similar approach, but attempts routing with the dynamic link-state graph G'
# This is the set of edges that hold a Bell state during a given timeslot
# for example, in a graph in which edges are generated with probability 0.7, and have edge-weight 1
graph_prime = nx.Graph()
graph_prime.add_edges_from([(u,v,{'weight':1}) for (u,v) in graph.edges() if random.random()<0.7])
if set(users) <= set(nx.node_connected_component(graph_prime,users[0])):
    print("solution found")
    routing_solution_tree = steiner_tree(graph_prime,users,weight = 'weight',method = 'pcst')
    plot_graph_routing_solution(graph=graph_prime,
                                routing_solution = routing_solution_tree,
                                users = users,
                                source = None)
else:
    print('no solution')
# Example is for large p and one timeslot of generation
# we generally consider when p << 1 and many rounds of  Bell state generation (edge in G') are required
# we also assume an edge will be discarded after 'Qc' timeslots

We can run a (short-ish) simulation, of the four protocols (sp-s, sp-t, mp-s, mp-t).\
 Default network assumptions are p =0.1 6x6 grid topology, and w, Delta taken from the paper. We consider a memory cutoff for values swept over qc in [1,20]. Results are averaged over 30 sets (S_reps) of users S which require a GHZ state. Each set consisting of four |S|=4 randomly selected nodes from the graph

In [ ]:
import os
import random

from qmcs.src.scatter_script import setup_args_scatter,default_params
from qmcs.src.plotting_script import plot_scatter_data
from qmcs.src.simulate import run_experiment
from qmcs.src.simulate import random_string

sim_params = default_params()
sim_params['str_id']  = random_string()
sim_params['experiment_name'] = "scatter"
sim_params['timesteps'] =100
sim_params['reps'] =30
sim_params['S_reps'] =30
for key,value in sim_params.items():
    print(key,value)
    
args_list,sim_params = setup_args_scatter(sim_params = sim_params)
random.shuffle(args_list)  # randomise dorder just to makes timing estimate more accurate
run_experiment(args_list, sim_params,save_dir=os.getcwd()+"//results")
plot_scatter_data(fig_str=sim_params['str_id'], save_figure=True,save_dir = os.getcwd()+"//figures")

We can extract some features of the routing solutions selected. E.g. the number of edges, and age of edges in the routing solution. These features can be used to describe why GHZ states are distributed at a given fidelity

In [ ]:
from qmcs.src.plotting_script import plot_scatter_other
str_id = sim_params['str_id'] 
for data_y in ['tree_sizes','tau_data']:
    plot_scatter_other(fig_str=str_id,
                    save_dir= os.getcwd()+"//figures",
                    x_param ='rate',
                    y_param = data_y)


We can consider the distribution rate achieved by the protocols with increasing distance between the users. We do this by putting four users in the corners of an MxM grid graph and simulating the distribution rate for increasing M. For this we assume p=0.5 and require the average GHZ fidelity > 2/3. For each distance we optimise Q_c such that the GHZ states are distributed with the maximum distribution rate while achieving above this min fidelity requirement.

In [ ]:
from qmcs.src.plotting_helpers import random_string
import os
import numpy as np
from qmcs.src.distance_script import run_distance_results
from qmcs.src.plotting_script import plot_distance_data
import numpy as np

sim_params = {"timesteps": 100, "reps": 30, "S": 4, "S_reps": 1}
sim_params["p_edge"] = 0.5
sim_params['funcs'] =['sp-s','sp-t','mp-s','mp-t']
sim_params['distance'] = np.arange(3,8)
sim_params['average_f_min'] = 2/3
 # f_min for average GHZ state fidelity in distance figure 
 # To set min fidelity of each GHZ state, set ('f_min')
sim_params['qc_max'] = 20
sim_params['str_id'] = random_string()

sim_params['experiment_name'] = 'distance'
run_distance_results(sim_params =sim_params, nice_plots = False)
plot_distance_data(sim_params['str_id'],
                save_figure=True,
                save_dir = os.getcwd()+"//figures",
                fig_axis = {'x':[3,7],'y':[1e-3,1]})